In [1]:
# Install required packages (force uninstall first to avoid conflicts)
%pip uninstall openai -y
%pip install openai==0.28.1

# Verify installation
import openai
print(f"OpenAI version: {openai.__version__}")
print("✅ Correct version installed!")

Found existing installation: openai 1.106.1
Uninstalling openai-1.106.1:
  Successfully uninstalled openai-1.106.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 2.1 MB/s eta 0:00:00
OpenAI version: 0.28.1
✅ Correct version installed!


In [3]:
# Import required libraries
import openai
import json
import os
from datetime import datetime
from typing import List, Dict, Any, Optional

In [13]:
# Groq API Configuration
# Groq API key configured


# Configure OpenAI client for Groq (compatible with openai==0.28.1)
openai.api_key = GROQ_API_KEY
openai.api_base = "https://api.groq.com/openai/v1"

In [14]:
class ConversationManager:
    """
    Manages conversation history with summarization and truncation capabilities.
    """

    def __init__(self, max_messages: int = 10, max_chars: int = 1000, summarize_every: int = 3):
        self.history: List[Dict[str, str]] = []
        self.max_messages = max_messages
        self.max_chars = max_chars
        self.summarize_every = summarize_every
        self.message_count = 0
        self.summary_count = 0

    def add_message(self, role: str, content: str) -> None:
        """Add a message to conversation history."""
        self.history.append({
            "role": role,
            "content": content,
            "timestamp": datetime.now().isoformat()
        })
        self.message_count += 1

        print(f" Added {role} message: {content[:50]}...")
        print(f" Total messages: {self.message_count}")

    def should_summarize(self) -> bool:
        """Check if conversation should be summarized."""
        return self.message_count > 0 and self.message_count % (self.summarize_every * 2) == 0

    def summarize_conversation(self) -> str:
        """Summarize the conversation using Groq API."""
        if not self.history:
            return "No conversation to summarize."

        # Format conversation for summarization
        conversation_text = "\n".join([
            f"{msg['role']}: {msg['content']}" for msg in self.history
        ])

        prompt = f"""
Summarize the following conversation while preserving key information:

Conversation:
{conversation_text}

Provide a concise summary that captures:
- Main topics discussed
- Key decisions made
- Important details mentioned

Summary:
"""

        try:
            response = openai.ChatCompletion.create(
                model="llama-3.1-8b-instant",
                messages=[{"role": "user", "content": prompt}],
                max_tokens=200,
                temperature=0.3
            )

            summary = response.choices[0].message.content
            self.summary_count += 1

            # Replace history with summary
            self.history = [{
                "role": "assistant",
                "content": f"[SUMMARY #{self.summary_count}] {summary}",
                "timestamp": datetime.now().isoformat()
            }]
            self.message_count = 1

            print(f" Conversation summarized! Summary #{self.summary_count}")
            return summary

        except Exception as e:
            print(f" Summarization failed: {e}")
            return "Summarization failed."

    def truncate_by_messages(self, keep_last: Optional[int] = None) -> None:
        """Keep only the last N messages."""
        if keep_last is None:
            keep_last = self.max_messages

        if len(self.history) > keep_last:
            removed_count = len(self.history) - keep_last
            self.history = self.history[-keep_last:]
            self.message_count = len(self.history)
            print(f" Truncated {removed_count} messages. Kept last {keep_last} messages.")
        else:
            print(f" No truncation needed. Current messages: {len(self.history)}")

    def truncate_by_chars(self, max_chars: Optional[int] = None) -> None:
        """Keep messages within character limit."""
        if max_chars is None:
            max_chars = self.max_chars

        current_chars = sum(len(msg['content']) for msg in self.history)

        if current_chars > max_chars:
            # Remove oldest messages until under limit
            while current_chars > max_chars and len(self.history) > 1:
                removed_msg = self.history.pop(0)
                current_chars -= len(removed_msg['content'])

            self.message_count = len(self.history)
            print(f" Truncated by character limit. Current chars: {current_chars}/{max_chars}")
        else:
            print(f" No character truncation needed. Current chars: {current_chars}/{max_chars}")

    def get_formatted_history(self) -> List[Dict[str, str]]:
        """Get conversation history formatted for API calls."""
        return [{"role": msg["role"], "content": msg["content"]} for msg in self.history]

    def display_history(self) -> None:
        """Display current conversation history."""
        print("\n📋 Current Conversation History:")
        print("=" * 50)

        if not self.history:
            print("No messages yet.")
            return

        for i, msg in enumerate(self.history, 1):
            print(f"{i}. [{msg['role'].upper()}] {msg['content'][:100]}{'...' if len(msg['content']) > 100 else ''}")

        print("\n Stats:")
        print(f"   Total messages: {self.message_count}")
        print(f"   Summaries created: {self.summary_count}")
        print(f"   Total characters: {sum(len(msg['content']) for msg in self.history)}")
        print("=" * 50)

print("✅ ConversationManager class defined successfully!")

✅ ConversationManager class defined successfully!


### Task 1 Demonstration


In [15]:
# Demo 1: Basic conversation management
print("🎯 Demo 1: Basic Conversation Management")
print("=" * 60)

# Create conversation manager
conv_mgr = ConversationManager(max_messages=8, summarize_every=3)

# Add sample conversation turns
sample_conversation = [
    ("user", "Hi, I need help with my order #12345"),
    ("assistant", "I'd be happy to help! What seems to be the issue with your order?"),
    ("user", "The package was delivered to the wrong address"),
    ("assistant", "I apologize for the inconvenience. Let me check your order details."),
    ("user", "The tracking shows it was delivered to 123 Main St, but I live at 456 Oak Ave"),
    ("assistant", "I see the issue. The shipping address was incorrectly entered. Let me arrange for redelivery."),
    ("user", "When can I expect the redelivery?"),
    ("assistant", "I'll schedule it for tomorrow between 9 AM and 5 PM. You'll receive a confirmation email shortly.")
]

# Add messages to conversation
for role, content in sample_conversation:
    conv_mgr.add_message(role, content)

# Display history
conv_mgr.display_history()


🎯 Demo 1: Basic Conversation Management
 Added user message: Hi, I need help with my order #12345...
 Total messages: 1
 Added assistant message: I'd be happy to help! What seems to be the issue w...
 Total messages: 2
 Added user message: The package was delivered to the wrong address...
 Total messages: 3
 Added assistant message: I apologize for the inconvenience. Let me check yo...
 Total messages: 4
 Added user message: The tracking shows it was delivered to 123 Main St...
 Total messages: 5
 Added assistant message: I see the issue. The shipping address was incorrec...
 Total messages: 6
 Added user message: When can I expect the redelivery?...
 Total messages: 7
 Added assistant message: I'll schedule it for tomorrow between 9 AM and 5 P...
 Total messages: 8

📋 Current Conversation History:
1. [USER] Hi, I need help with my order #12345
2. [ASSISTANT] I'd be happy to help! What seems to be the issue with your order?
3. [USER] The package was delivered to the wrong address
4. [A

## Task 2: JSON Schema Classification & Information Extraction

### Features to implement:
1. Create JSON schema for 5 personal information fields
2. Use OpenAI function calling with Groq API
3. Extract structured data from conversations
4. Validate outputs against schema


In [7]:
# JSON Schema for personal information extraction
classification_schema = {
    "type": "object",
    "properties": {
        "name": {
            "type": "string",
            "description": "Full name of the person"
        },
        "email": {
            "type": "string",
            "description": "Email address"
        },
        "phone": {
            "type": "string",
            "description": "Phone number"
        },
        "location": {
            "type": "string",
            "description": "City, state, or address"
        },
        "age": {
            "type": "integer",
            "description": "Age in years"
        }
    },
    "required": ["name", "email"]
}

print("✅ JSON Schema defined for personal information extraction:")
print(json.dumps(classification_schema, indent=2))

✅ JSON Schema defined for personal information extraction:
{
  "type": "object",
  "properties": {
    "name": {
      "type": "string",
      "description": "Full name of the person"
    },
    "email": {
      "type": "string",
      "description": "Email address"
    },
    "phone": {
      "type": "string",
      "description": "Phone number"
    },
    "location": {
      "type": "string",
      "description": "City, state, or address"
    },
    "age": {
      "type": "integer",
      "description": "Age in years"
    }
  },
  "required": [
    "name",
    "email"
  ]
}


In [8]:
def extract_personal_information(chat_message: str) -> Dict[str, Any]:
    """
    Extract personal information from a chat message using Groq API function calling.
    """
    try:
        response = openai.ChatCompletion.create(
            model="llama-3.3-70b-versatile",  # Using model that supports function calling
            messages=[{
                "role": "user",
                "content": f"Extract personal information from this message: {chat_message}"
            }],
            functions=[{
                "name": "extract_personal_info",
                "description": "Extract personal information from conversation",
                "parameters": classification_schema
            }],
            function_call={"name": "extract_personal_info"},
            temperature=0.1  # Low temperature for consistent extraction
        )

        # Parse the function call result
        function_call = response.choices[0].message.function_call
        if function_call and function_call.name == "extract_personal_info":
            extracted_data = json.loads(function_call.arguments)
            return extracted_data
        else:
            return {"error": "No function call returned"}

    except Exception as e:
        return {"error": f"Extraction failed: {str(e)}"}

def validate_extraction(extracted_data: Dict[str, Any]) -> Dict[str, Any]:
    """
    Validate extracted data against the schema.
    """
    validation_result = {
        "is_valid": True,
        "errors": [],
        "warnings": []
    }

    # Check required fields
    required_fields = classification_schema["required"]
    for field in required_fields:
        if field not in extracted_data or not extracted_data[field]:
            validation_result["errors"].append(f"Missing required field: {field}")
            validation_result["is_valid"] = False

    # Check field types
    properties = classification_schema["properties"]
    for field, value in extracted_data.items():
        if field in properties:
            expected_type = properties[field]["type"]
            actual_type = type(value).__name__

            if expected_type == "integer" and actual_type != "int":
                validation_result["errors"].append(f"Field '{field}' should be integer, got {actual_type}")
                validation_result["is_valid"] = False
            elif expected_type == "string" and actual_type != "str":
                validation_result["errors"].append(f"Field '{field}' should be string, got {actual_type}")
                validation_result["is_valid"] = False

    # Check email format (basic validation)
    if "email" in extracted_data and "@" not in str(extracted_data["email"]):
        validation_result["warnings"].append("Email format may be invalid")

    return validation_result

print(" Information extraction functions defined successfully!")

 Information extraction functions defined successfully!


### Task 2 Demonstration with Sample Chats


In [9]:
# Sample Chat 1: Complete information
print(" Demo 1: Sample Chat with Complete Information")
print("=" * 60)

sample_chat_1 = """
Hi, my name is John Smith. I'm 25 years old and live in New York City.
My email address is john.smith@example.com and you can reach me at 555-123-4567.
I'm interested in learning more about your services.
"""

print(f" Sample Chat 1:\n{sample_chat_1.strip()}")

# Extract information
print("\n Extracting information...")
extracted_1 = extract_personal_information(sample_chat_1)

print(f"\n Extracted Data:")
print(json.dumps(extracted_1, indent=2))

# Validate extraction
print("\n Validation Results:")
validation_1 = validate_extraction(extracted_1)
print(json.dumps(validation_1, indent=2))


 Demo 1: Sample Chat with Complete Information
 Sample Chat 1:
Hi, my name is John Smith. I'm 25 years old and live in New York City. 
My email address is john.smith@example.com and you can reach me at 555-123-4567. 
I'm interested in learning more about your services.

 Extracting information...

 Extracted Data:
{
  "age": 25,
  "email": "john.smith@example.com",
  "location": "New York City",
  "name": "John Smith",
  "phone": "555-123-4567"
}

 Validation Results:
{
  "is_valid": true,
  "errors": [],
  "warnings": []
}


In [16]:
# Sample Chat 2: Partial information
print("\n Demo 2: Sample Chat with Partial Information")
print("=" * 60)

sample_chat_2 = """
Hello, I'm Sarah Johnson from California. My phone number is 555-987-6543.
I'm 30 years old and my email is sarah.johnson@company.com.
Can you help me with my account?
"""

print(f" Sample Chat 2:\n{sample_chat_2.strip()}")

# Extract information
print("\n Extracting information...")
extracted_2 = extract_personal_information(sample_chat_2)

print(f"\n Extracted Data:")
print(json.dumps(extracted_2, indent=2))

# Validate extraction
print("\n Validation Results:")
validation_2 = validate_extraction(extracted_2)
print(json.dumps(validation_2, indent=2))



 Demo 2: Sample Chat with Partial Information
 Sample Chat 2:
Hello, I'm Sarah Johnson from California. My phone number is 555-987-6543. 
I'm 30 years old and my email is sarah.johnson@company.com. 
Can you help me with my account?

 Extracting information...

 Extracted Data:
{
  "age": 30,
  "email": "sarah.johnson@company.com",
  "location": "California",
  "name": "Sarah Johnson",
  "phone": "555-987-6543"
}

 Validation Results:
{
  "is_valid": true,
  "errors": [],
  "warnings": []
}


In [11]:
# Sample Chat 3: Minimal information
print("\n Demo 3: Sample Chat with Minimal Information")
print("=" * 60)

sample_chat_3 = """
Hi there! I'm Mike, 28 years old, and I live in Texas.
My contact email is mike.wilson@testmail.com and my phone is 555-456-7890.
I need help with my subscription.
"""

print(f" Sample Chat 3:\n{sample_chat_3.strip()}")

# Extract information
print("\n Extracting information...")
extracted_3 = extract_personal_information(sample_chat_3)

print(f"\n Extracted Data:")
print(json.dumps(extracted_3, indent=2))

# Validate extraction
print("\n Validation Results:")
validation_3 = validate_extraction(extracted_3)
print(json.dumps(validation_3, indent=2))


 Demo 3: Sample Chat with Minimal Information
 Sample Chat 3:
Hi there! I'm Mike, 28 years old, and I live in Texas. 
My contact email is mike.wilson@testmail.com and my phone is 555-456-7890. 
I need help with my subscription.

 Extracting information...

 Extracted Data:
{
  "age": 28,
  "email": "mike.wilson@testmail.com",
  "location": "Texas",
  "name": "Mike",
  "phone": "555-456-7890"
}

 Validation Results:
{
  "is_valid": true,
  "errors": [],
  "warnings": []
}


In [12]:
# Summary and Comparison of All Extractions
print("\n Summary: All Extraction Results Comparison")
print("=" * 70)

all_extractions = [
    ("Sample Chat 1", extracted_1, validation_1),
    ("Sample Chat 2", extracted_2, validation_2),
    ("Sample Chat 3", extracted_3, validation_3)
]

print("\n Extraction Results Summary:")
print("-" * 70)

for chat_name, extracted, validation in all_extractions:
    print(f"\n{chat_name}:")
    print(f"   Valid: {validation['is_valid']}")

    if validation['errors']:
        print(f"   Errors: {', '.join(validation['errors'])}")

    if validation['warnings']:
        print(f"    Warnings: {', '.join(validation['warnings'])}")

    # Show extracted fields
    fields_found = [f"{k}: {v}" for k, v in extracted.items() if k != 'error' and v]
    print(f"   Fields: {', '.join(fields_found)}")

print("\n" + "=" * 70)
print(" All 3 sample chats processed and validated against JSON schema")


 Summary: All Extraction Results Comparison

 Extraction Results Summary:
----------------------------------------------------------------------

Sample Chat 1:
   Valid: True
   Fields: age: 25, email: john.smith@example.com, location: New York City, name: John Smith, phone: 555-123-4567

Sample Chat 2:
   Valid: True
   Fields: age: 30, email: sarah.johnson@company.com, location: California, name: Sarah Johnson, phone: 555-987-6543

Sample Chat 3:
   Valid: True
   Fields: age: 28, email: mike.wilson@testmail.com, location: Texas, name: Mike, phone: 555-456-7890

 All 3 sample chats processed and validated against JSON schema
